In [1]:
pip install rrcf

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import rrcf
def load_data(dataFile: str):
  df = pd.read_csv(dataFile,sep=' ')
  df = df.astype(float)
  class_labels = df.iloc[:, -1:]
  class_labels = class_labels.to_numpy()
  df = df.iloc[: , :-1]
  df = df.to_numpy()
  return df, class_labels


def load_data_without_labels(dataFile: str):
  df = pd.read_csv(dataFile,sep=' ')
  df = df.astype(float)
  df = df.to_numpy()
  return df


def AAR(df: pd.DataFrame):
    #df.drop(0,axis=1,inplace=True)
    df.sort_values(by=0,ascending=False,inplace=True)
    df=df.reset_index(drop=True)
    AAR = np.mean(df.index[df[1] == 1].tolist())
    return AAR


In [55]:
input_path = '../io_directory/'
D0,D0_class_labels = load_data(input_path+'D0.csv')

In [56]:
nD0 = len(D0)

In [57]:
# Set forest parameters
num_trees = 100
tree_size = 256
sample_size_range = (nD0 // tree_size, tree_size)

# Construct forest
forest = []
while len(forest) < num_trees:
    # Select random subsets of points uniformly
    ixs = np.random.choice(nD0, size=sample_size_range,
                           replace=False)
    # Add sampled trees to forest
    trees = [rrcf.RCTree(D0[ix], index_labels=ix)
             for ix in ixs]
    forest.extend(trees)

In [58]:
avg_codisp_D0 = {}
avg_codisp_D0=pd.Series(0.0, index=np.arange(nD0))
for index, point in enumerate(D0):
    # For each tree in the forest...
    for tree in forest:
        #print(index,point)
        # If tree is above permitted size...
        #if len(tree.leaves) > tree_size:
            # Drop the oldest point (FIFO)
            #tree.forget_point(index+10000 - tree_size)
        # Insert the new point into the tree
        #tree.insert_point(point, index=index)
        # Compute codisp on the new point...
        
        new_codisp = tree.codisp(tree.query(point))
        # And take the average over all trees
        avg_codisp_D0[index] += new_codisp / num_trees

In [59]:
avg_codisp_D0

0        21.577789
1        67.208798
2        10.615479
3        14.691587
4        10.518025
           ...    
45214    15.818172
45215    19.832099
45216    56.726851
45217    52.566254
45218    36.936341
Length: 45219, dtype: float64

In [60]:
AAR(pd.concat([avg_codisp_D0,pd.DataFrame(D0_class_labels)],axis=1,ignore_index=True))

4549.08988764045

In [61]:
deltaD,deltaD_class_labels = load_data(input_path+'deltaD1.csv')
ndeltaD=len(deltaD)

In [62]:
avg_codisp = {}
avg_codisp=pd.Series(0.0, index=np.arange(nD0,nD0+ndeltaD))
avg_codisp
for index, point in enumerate(deltaD):
    # For each tree in the forest...
    index += nD0
    for tree in forest:
        #print(index,point)
        # If tree is above permitted size...
        #if len(tree.leaves) > tree_size:
            # Drop the oldest point (FIFO)
            #tree.forget_point(index+10000 - tree_size)
        # Insert the new point into the tree
        tree.insert_point(point, index=index)
        # Compute codisp on the new point...
        new_codisp = tree.codisp(index)
        # And take the average over all trees
        if not index in avg_codisp:
            avg_codisp[index] = 0
        avg_codisp[index] += new_codisp / num_trees

KeyboardInterrupt: 

In [ ]:
avg_codisp

In [ ]:
#labels  = pd.DataFrame(deltaD_class_labels,index=range(45219,45699))

In [ ]:
#df=pd.concat([avg_codisp,pd.DataFrame(deltaD_class_labels,index=range(45219,45699))],axis=1,ignore_index=True)

In [ ]:
#df[1].to_list()

In [ ]:
AAR(pd.concat([avg_codisp,pd.DataFrame(deltaD_class_labels,index=range(45219,45699))],axis=1,ignore_index=True))

In [ ]:
df

In [ ]:
avg_codisp_D0 = {}
avg_codisp_D0=pd.Series(0.0, index=np.arange(nD0))
for index, point in enumerate(D0):
    # For each tree in the forest...
    for tree in forest:
        #print(index,point)
        # If tree is above permitted size...
        #if len(tree.leaves) > tree_size:
            # Drop the oldest point (FIFO)
            #tree.forget_point(index+10000 - tree_size)
        # Insert the new point into the tree
        #tree.insert_point(point, index=index)
        # Compute codisp on the new point...
        
        new_codisp = tree.codisp(tree.query(point))
        # And take the average over all trees
        avg_codisp_D0[index] += new_codisp / num_trees

In [ ]:
avg_codisp_D0

In [ ]:
AAR(pd.concat([avg_codisp_D0,pd.DataFrame(D0_class_labels)],axis=1,ignore_index=True))